In [2]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
learning_rate = 0.001
batch_size = 32
epochs = 10


In [4]:
(train_ds, valid_ds), info = tfds.load("food101", split=["train", "validation"], as_supervised=True, with_info=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteBES7PL/food101-train.tfrecord*...:   0%|          …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteBES7PL/food101-validation.tfrecord*...:   0%|     …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [5]:
def preprocess_dataset(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.mobilenet.preprocess_input(image)
    binary_label = tf.where(tf.equal(label, 55), 1, 0)
    return image, binary_label


In [6]:
train_ds = train_ds.map(preprocess_dataset)
valid_ds = valid_ds.map(preprocess_dataset)

In [7]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


17225924/17225924 [==============================] - 1s 0us/step


In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [14]:
history = model.fit(
    train_ds.batch(batch_size),
    validation_data=valid_ds.batch(batch_size),
    epochs=epochs
)

Epoch 1/10
2368/2368 [==============================] - 235s 99ms/step - loss: 0.0173 - accuracy: 0.9950 - val_loss: 0.0195 - val_accuracy: 0.9946
Epoch 2/10
2368/2368 [==============================] - 205s 86ms/step - loss: 0.0146 - accuracy: 0.9957 - val_loss: 0.0213 - val_accuracy: 0.9946
Epoch 3/10
2368/2368 [==============================] - 203s 85ms/step - loss: 0.0118 - accuracy: 0.9967 - val_loss: 0.0243 - val_accuracy: 0.9941
Epoch 4/10
2368/2368 [==============================] - 238s 100ms/step - loss: 0.0091 - accuracy: 0.9972 - val_loss: 0.0254 - val_accuracy: 0.9939
Epoch 5/10
2368/2368 [==============================] - 237s 99ms/step - loss: 0.0076 - accuracy: 0.9976 - val_loss: 0.0239 - val_accuracy: 0.9944
Epoch 6/10
2368/2368 [==============================] - 202s 85ms/step - loss: 0.0061 - accuracy: 0.9980 - val_loss: 0.0231 - val_accuracy: 0.9945
Epoch 7/10
2368/2368 [==============================] - 203s 85ms/step - loss: 0.0045 - accuracy: 0.9986 - val_loss: 

## Now we will build a model without using MobileNet

In [15]:
MAX_SIDE_LEN = 128
HOT_DOG_CLASS = 55
train_ds = train_ds.map(
    lambda image, label: (tf.cast(tf.image.resize(image, [MAX_SIDE_LEN, MAX_SIDE_LEN]), dtype=tf.int32), tf.cast(label == HOT_DOG_CLASS, dtype=tf.int32))
)
valid_ds = valid_ds.map(
    lambda image, label: (tf.cast(tf.image.resize(image, [MAX_SIDE_LEN, MAX_SIDE_LEN]), dtype=tf.int32), tf.cast(label == HOT_DOG_CLASS, dtype=tf.int32))
)

In [16]:
train_hd_size, valid_hd_size = 750,250
train_hotdogs = train_ds.filter(lambda image, label: label==1)
train_nothotdogs = train_ds.filter(lambda image, label: label==0)

In [17]:
valid_hotdogs = valid_ds.filter(lambda image, label: label==1)
valid_nothotdogs = valid_ds.filter(lambda image, label: label==0)

In [18]:
batch_size = 16
train_ds = tf.data.Dataset.sample_from_datasets([train_hotdogs, train_nothotdogs],
                                                [0.5,0.5],
                                                stop_on_empty_dataset=True)
train_ds = train_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
valid_ds = tf.data.Dataset.sample_from_datasets([valid_hotdogs, valid_nothotdogs],
                                                [0.5,0.5],
                                                stop_on_empty_dataset=True)
valid_ds = valid_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [20]:
random.seed(0)
model = models.Sequential()
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(128, (3, 3), activation = 'relu', input_shape= [MAX_SIDE_LEN, MAX_SIDE_LEN, 3]))
model.add(layers.MaxPooling2D(2 ,2))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D(2 ,2))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1))

In [21]:
lr = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])


In [22]:
epochs = 50
history = model.fit(
    train_ds,
    validation_data = valid_ds,
    epochs=epochs,
    verbose=1
)

Epoch 1/50
1/1 [==============================] - 51s 51s/step - loss: 0.6931 - accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 25ms/step - loss: 0.6931 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 20ms/step - loss: 0.6930 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 20ms/step - loss: 0.6930 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 21ms/step - loss: 0.6929 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 20ms/step - loss: 0.6929 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 20ms/step - loss: 0.6928 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 24ms/step - loss: 0.6928 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 21ms/step - loss: 0.6927 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 21ms/step - loss: 0.6927 - accuracy: 1.0000
Epoch 11/50
1/1 [============